In [1]:
pip install -U transformers peft torch scikit-learn datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 120.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.0/557.0 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 145.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
import os
from google import genai
import numpy as np
from google.colab import userdata

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import PeftModel
from sklearn.metrics import f1_score, roc_auc_score

from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

## 사용자 질문 판단 (필수)

In [3]:
# API 키 설정
client = genai.Client(
    api_key=userdata.get('GOOGLE_API_KEY')
)

# 카테고리 목록
CATEGORIES = [
    "경제활동 · 상품 · 상거래",
    "기술 · 과학",
    "미용 · 관광 · 식음료",
    "엔터테이먼트 · 오락",
    "여행 · 여가 · 취미",
    "인문 · 사회",
    "주거 · 생활 · 사람관계"
]


def classify_text(text):
    """
    AI 답변을 Gemini API를 사용하여 카테고리로 분류합니다.
    """
    try:
        # 프롬프트 구성
        prompt = f"""다음 텍스트를 아래 7가지 카테고리 중 가장 적합한 하나로 분류해주세요.
반드시 카테고리 이름만 정확히 출력하고, 다른 설명은 추가하지 마세요.

카테고리 목록:
1. 경제활동 · 상품 · 상거래
2. 기술 · 과학
3. 미용 · 관광 · 식음료
4. 엔터테이먼트 · 오락
5. 여행 · 여가 · 취미
6. 인문 · 사회
7. 주거 · 생활 · 사람관계

분류할 텍스트:
{text}

가장 적합한 카테고리:"""

        # API 호출 방식 변경 (models.generate_content 사용)
        response = client.models.generate_content(
            model="gemini-2.5-flash-lite",  # 최신 모델 권장
            contents=prompt
        )

        category = response.text.strip()

        # 응답 확인 로직 (기존과 동일)
        if category in CATEGORIES:
            return category
        else:
            for cat in CATEGORIES:
                if cat in category or category in cat:
                    return cat
            return f"분류 실패: {category}"

    except Exception as e:
        return f"오류 발생: {str(e)}"


def classify_batch(texts):
    results = []
    for i, text in enumerate(texts):
        category = classify_text(text)
        results.append({
            "index": i,
            "text": text[:100] + "..." if len(text) > 100 else text,
            "category": category
        })
    return results


if __name__ == "__main__":
    print("분류할 텍스트를 입력하세요:")
    sample_text = input()

    print("=" * 60)
    print("단일 텍스트 분류 예시")
    print("=" * 60)
    print(f"텍스트: {sample_text}")
    print(f"분류 결과: {classify_text(sample_text)}")
    print()

    results = classify_batch(sample_text)
    for result in results:
        print(f"[{result['index'] + 1}] {result['text']}")
        print(f"    → 카테고리: {result['category']}\n")

분류할 텍스트를 입력하세요:
달에 아파트를 짓고싶어
단일 텍스트 분류 예시
텍스트: 달에 아파트를 짓고싶어
분류 결과: 주거 · 생활 · 사람관계

[1] 달
    → 카테고리: 기술 · 과학

[2] 에
    → 카테고리: 인문 · 사회

[3]  
    → 카테고리: 주거 · 생활 · 사람관계

[4] 아
    → 카테고리: 주거 · 생활 · 사람관계

[5] 파
    → 카테고리: 경제활동 · 상품 · 상거래

[6] 트
    → 카테고리: 엔터테이먼트 · 오락

[7] 를
    → 카테고리: 오류 발생: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash-lite\nPlease retry in 32.57114375s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type':

## 라벨 설정 (필수)

In [4]:
labels = ['linguistic_acceptability', 'consistency', 'interestingness', 'unbias',
          'harmlessness', 'no_hallucination', 'understandability', 'sensibleness', 'specificity']

id2label = {idx: label for idx, label in enumerate(labels)}
label2id = {label: idx for idx, label in enumerate(labels)}

## 모델 호출 후 sample 추론해보기 (필수)

In [5]:
# =========================
# 1) 설정 (본인 값으로 교체)
# =========================
BASE_MODEL_ID = "team-lucid/deberta-v3-base-korean"

# 허깅페이스에 push한 LoRA 어댑터 repo id (학습 코드의 hub_model_id)
LORA_ADAPTER_REPO = "yj512/deberta-korean-multilabel_base-lora2"

# 멀티라벨 클래스 정보 (학습 때 쓰던 것과 동일해야 함)
# - labels 리스트 / id2label / label2id 를 학습 때와 동일하게 세팅하세요.
# 예시:
# labels = ["A", "B", "C"]
# id2label = {0:"A", 1:"B", 2:"C"}
# label2id = {"A":0, "B":1, "C":2}

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if device == "cuda" else torch.float32

# =========================
# 2) 토크나이저 & 베이스 모델 로드
# =========================
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID, use_fast=True)

base_model = AutoModelForSequenceClassification.from_pretrained(
    BASE_MODEL_ID,
    problem_type="multi_label_classification",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
    torch_dtype=dtype,
).to(device)

# =========================
# 3) LoRA 어댑터 로드해서 결합
# =========================
model = PeftModel.from_pretrained(base_model, LORA_ADAPTER_REPO)
model.eval()

# =========================
# 4) 멀티라벨 추론 함수
# =========================
@torch.no_grad()
def predict_texts(texts, threshold=0.5, max_length=256, batch_size=32):
    """
    texts: List[str]
    return:
      - probs: (N, C) float
      - pred:  (N, C) int (0/1)
    """
    all_probs = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        enc = tokenizer(
            batch,
            padding=True,
            truncation=True,
            max_length=max_length,
            return_tensors="pt",
        ).to(device)

        logits = model(**enc).logits  # (B, C)
        probs = torch.sigmoid(logits).detach().cpu().numpy()
        all_probs.append(probs)

    probs = np.concatenate(all_probs, axis=0)
    pred = (probs >= threshold).astype(int)
    return probs, pred

# =========================
# 5) 테스트 예시 (단일)
# =========================
sample_text = "이 문장은 멀티라벨 분류 테스트용입니다."
probs, pred = predict_texts([sample_text], threshold=0.5)

print("probs:", probs[0])
print("pred :", pred[0])
print("labels predicted:", [id2label[i] for i,v in enumerate(pred[0]) if v == 1])

tokenizer_config.json:   0%|          | 0.00/390 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at team-lucid/deberta-v3-base-korean and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


adapter_config.json: 0.00B [00:00, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/10.8M [00:00<?, ?B/s]

probs: [0.9614 0.913  0.8926 0.9893 0.9976 0.779  0.7944 0.852  0.921 ]
pred : [1 1 1 1 1 1 1 1 1]
labels predicted: ['linguistic_acceptability', 'consistency', 'interestingness', 'unbias', 'harmlessness', 'no_hallucination', 'understandability', 'sensibleness', 'specificity']


## test data set 있으면 추론해보기 (선택)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from datasets import load_from_disk

encoded_test=load_from_disk("/content/drive/MyDrive/멋쟁이사자처럼/data/encoded_test")

In [ ]:
encoded_test

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

def multi_label_metrics_from_logits(logits, labels, threshold=0.5):
    # logits: (N, C), labels: (N, C)
    probs = torch.sigmoid(torch.from_numpy(logits))
    y_pred = (probs.numpy() >= threshold).astype(int)

    # labels는 0/1 멀티핫이라고 가정
    f1_micro = f1_score(labels, y_pred, average="micro")
    roc_auc_micro = roc_auc_score(labels, y_pred, average="micro")
    return {"f1_micro": f1_micro, "roc_auc_micro": roc_auc_micro}

@torch.no_grad()
def evaluate_encoded_val(encoded_test, tokenizer, model, batch_size=128, threshold=0.5):
    # 1) Dataset을 torch 텐서 포맷으로 (labels 포함)
    cols = ["input_ids", "attention_mask", "token_type_ids", "labels"]
    existing_cols = [c for c in cols if c in encoded_test.column_names]
    ds = encoded_test.with_format("torch", columns=existing_cols)

    # 2) padding/collate
    collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

    loader = DataLoader(ds, batch_size=batch_size, shuffle=False, collate_fn=collator)

    all_logits = []
    all_labels = []

    model.eval()
    for batch in loader:
        # labels는 forward에 넣지 말고 따로 빼는 게 안전
        labels = batch.pop("labels")

        # device 이동
        batch = {k: v.to(device) for k, v in batch.items()}
        labels = labels.to(device)

        # DeBERTa는 token_type_ids가 있을 수 있음 (batch에 있으면 자동 포함)
        outputs = model(**batch)
        logits = outputs.logits  # (B, C)

        all_logits.append(logits.detach().cpu().numpy())
        # 멀티라벨이면 float/ int 상관없지만, metric 계산은 numpy로
        all_labels.append(labels.detach().cpu().numpy())

    all_logits = np.concatenate(all_logits, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    return multi_label_metrics_from_logits(all_logits, all_labels, threshold=threshold)

# ✅ 실행 예시
metrics = evaluate_encoded_val(encoded_test, tokenizer, model, batch_size=128, threshold=0.5)
print(metrics)

In [ ]:
def weighted_score(
    probs,
    weights,
    normalize=True,
):
    """
    probs   : (C,) or (N, C) array, sigmoid 결과
    weights : (C,) array, 라벨별 중요도
    normalize : 가중치 합으로 나눌지 여부

    return  : scalar 또는 (N,) score
    """
    probs = np.asarray(probs)
    weights = np.asarray(weights)

    score = probs @ weights  # dot product

    if normalize:
        score = score / weights.sum()

    return score


In [ ]:
# 임의
probs = np.array([0.9285, 0.8320, 0.8661, 0.9722, 0.9882,
                  0.7247, 0.8570, 0.7766, 0.8988])

# 임의
weights = np.array([1.0, 0.8, 1.2, 1.5, 2.0, 0.5, 1.0, 0.7, 1.3])

score = weighted_score(probs, weights)*10
print(score)  # 0~10 사이

## 가중치 설정 (필수)



In [6]:
# labels = ['linguistic_acceptability', 'consistency', 'interestingness', 'unbias','harmlessness', 'no_hallucination', 'understandability', 'sensibleness', 'specificity']

Science_Weights = np.array([
    1.5,  # linguistic_acceptability
    1.5,  # consistency
    0.5,  # interestingness
    0.5,  # unbias
    0.5,  # harmlessness
    2.0,  # no_hallucination
    2.0,  # understandability
    1.5,  # sensibleness
    1.5,  # specificity
])

Life_Weights = np.array([
    1.0,  # linguistic_acceptability
    0.5,  # consistency
    1.0,  # interestingness
    1.0,  # unbias
    1.5,  # harmlessness
    1.0,  # no_hallucination
    0.5,  # understandability
    1.5,  # sensibleness
    0.5,  # specificity
])



In [7]:
# 계산 방식
def weighted_score_from_probs(
    probs,
    weights,
    threshold=0.5,
    normalize=True,
):
    """
    probs   : (C,) np.ndarray  — sigmoid 결과
    weights : (C,) np.ndarray  — 라벨별 중요도
    """
    probs = np.asarray(probs)
    weights = np.asarray(weights)

    mask = probs >= threshold
    if not mask.any():
        return 0.0

    score = (probs[mask] * weights[mask]).sum()

    if normalize:
        score = score / weights[mask].sum()

    score = round(float(score), 4)*10

    return float(score)


In [8]:
# 가중치 선택 코드
def select_weights_by_category(category: str):
    """
    카테고리에 따라 가중치 선택
    """
    if category == "기술 · 과학":
        return Science_Weights
    elif category == "주거 · 생활 · 사람관계":
        return Life_Weights
    else:
        # 기본값: 기술·과학 가중치
        return Science_Weights

## 최종 실험 코드

In [9]:
@torch.no_grad()
def infer_with_weighted_score(
    text,
    tokenizer,
    model,
    classify_fn,          # classify_text 함수
    threshold=0.5,
    max_length=256,
):
    model.eval()

    # 카테고리 분류
    category = classify_fn(text)

    # 카테고리에 따른 가중치 선택
    label_weights = select_weights_by_category(category)

    # 모델 추론
    enc = tokenizer(
        text,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt",
    ).to(model.device)

    logits = model(**enc).logits[0]   # (C,)
    probs = torch.sigmoid(logits).cpu().numpy()
    pred = (probs >= threshold).astype(int)

    # 4가중치 점수 계산
    score = weighted_score_from_probs(
        probs=probs,
        weights=label_weights,
        threshold=threshold,
    )

    return {
        "category": category,
        "weights_used": "Life_Weights" if category == "주거 · 생활 · 사람관계" else "Science_Weights",
        "probs": probs,
        "pred": pred,
        "score": score,
    }


## 기술과학

In [11]:
text = """
단순히 바다 밑에 건물을 짓는 것을 넘어, 수압과 산소 공급 등 **극한 환경을 극복하고 인간이 장기간 자급자족하며 거주할 수 있도록 설계된 미래형 도시**를 뜻합니다.

해저 기지나 심해 호텔보다 큰 규모로, 에너지 생산과 식량 조달 시스템까지 갖춘 **하나의 완전한 생태계**를 구축하는 것이 핵심입니다.

더 자세한 기술적 원리나 현재 진행 중인 실제 프로젝트 사례가 궁금하신가요?
가장 중요한 조건은 **엄청난 수압을 견디는 고강도 구조체**, **독립적인 산소 공급 및 이산화탄소 제거 시스템**, 그리고 **안정적인 에너지원** 확보입니다.

현재 기술로 소규모 해저 기지나 호텔 운영은 가능하지만, 수천 명이 상주하는 대규모 도시는 경제성과 비상 대피 기술 문제로 아직 연구 단계에 있습니다.

혹시 심해 도시 건설에서 가장 큰 걸림돌인 '수압'을 어떻게 해결하는지 구체적으로 알아볼까요?
심해도시 실현이 어려운 이유는 지상보다 **수백 배 높은 수압**을 견디는 천문학적인 건설 비용과 사고 시 **대피가 거의 불가능한 안전상의 위험** 때문입니다.

또한 금속을 부식시키는 염분과 햇빛이 들지 않는 어두운 환경이 **거주자의 정신 건강과 생태계 파괴**에 미칠 악영향도 큰 걸림돌로 지목됩니다.

혹시 이 모든 위험을 감수하고도 인류가 왜 심해에 관심을 가지는지, 그 잠재력에 대해 더 궁금하신가요?
심해도시 실현이 어려운 이유는 지상보다 **수백 배 높은 수압**을 견디는 천문학적인 건설 비용과 사고 시 **대피가 거의 불가능한 안전상의 위험** 때문입니다.

또한 금속을 부식시키는 염분과 햇빛이 들지 않는 어두운 환경이 **거주자의 정신 건강과 생태계 파괴**에 미칠 악영향도 큰 걸림돌로 지목됩니다.

혹시 이 모든 위험을 감수하고도 인류가 왜 심해에 관심을 가지는지, 그 잠재력에 대해 더 궁금하신가요?
기술적으로는 소수 인원의 단기 거주가 가능하지만, 대규모 도시를 유지하기 위한 천문학적인 건설·유지 비용과 사고 시 구조가 불가능에 가깝다는 안전 문제가 가장 큰 현실적 장벽입니다.

지상이나 우주 도시 후보지에 비해 경제적 효용성이 낮고, 심해의 극한 환경에서 발생할 수 있는 돌발 변수를 통제하기에 현재의 인프라와 자본 투입은 위험 부담이 너무 큽니다.

심해도시 대신 현재 실현 가능성이 더 높다고 평가받는 '해상 부유 도시' 프로젝트에 대해 알아볼까요?

"""

result = infer_with_weighted_score(
    text=text,
    tokenizer=tokenizer,
    model=model,
    classify_fn=classify_text,
    threshold=0.6
)

print("probs :", result["probs"])
print("pred  :", result["pred"])
print("score :", result["score"])


probs : [0.9453 0.944  0.9517 0.9634 0.9272 0.814  0.3696 0.937  0.9688]
pred  : [1 1 1 1 1 1 0 1 1]
score : 9.202


In [27]:
text = """
‘심해도시’는 단순히 바다 밑에 도시를 짓는다는 뜻보다는, 심해 환경에서 인간 활동이 가능한 거주·연구·산업 공간 개념을 말해. 현재는 공상과학적 요소가 크고, 실제로는 해저 기지·연구소 수준의 개념에 가까워.
가장 중요한 건 **압력 견디는 구조, 산소·에너지 공급, 장기 거주 시 인체 안전**이야. 지금 기술로는 잠수정·해저 연구기지처럼 **단기 체류나 제한적 거주**는 가능하지만, 상시 생활은 아직 어려워.
심해는 극한 압력·고비용 건설·사고 시 복구 불가능성 때문에 위험 부담이 커. 또 지상·우주보다 얻는 이익이 명확하지 않아 투자 대비 효용이 낮다고 보는 시각이 많아.
A4. 가장 큰 이유는 기술 부족보다는 비용과 안전 문제야. 기술은 부분적으로 가능하지만, 사고 시 치명적이고 유지비가 너무 커서 현실성이 떨어져.
"""

result = infer_with_weighted_score(
    text=text,
    tokenizer=tokenizer,
    model=model,
    classify_fn=classify_text,
    threshold=0.6
)

print("probs :", result["probs"])
print("pred  :", result["pred"])
print("score :", result["score"])

probs : [0.8276 0.958  0.948  0.9897 0.9937 0.8867 0.3105 0.9097 0.9585]
pred  : [1 1 1 1 1 1 0 1 1]
score : 9.179


In [ ]:
text = """
**‘심해도시’**는 보통 쿠키런 같은 게임에서 나오는 심해(깊은 바다) 속 판타지 도시를 가리키는 말이에요.
현실적으로는 사람이 살 수 없는 극한 수압의 심해(2km 이상)에 도시를 짓는다는 뜻은 거의 아니고, 그냥 바다 밑/바다 위에 도시 만든다는 SF적 상상 정도로 이해하면 돼요. 😄
가장 중요한 조건 (순위순):

극한 압력 견디는 구조 (1m당 0.1기압↑)
산소 공급 + CO₂ 제거
심리적 고립 극복

현재 기술로는 수백 미터에서 장기 거주 불가능하고, 30~50m 정도가 실질적 한계선입니다. 😅
심해도시가 어려운 진짜 이유 (3줄 요약):

1km 깊이만 가도 압력이 100기압 → 현재 기술로 만들 수 있는 구조물 비용이 천문학적
에너지·식량·폐기물·정신 건강 문제 모두 해결 불가 수준
육지/해상 도시보다 비용 대비 효율이 극도로 떨어져서 실현 가능성 거의 없음

가장 큰 이유는 비용입니다.
기술적으로도 아직 부족하지만, 그걸 극복하려면 현재 비용이 수십~수백조 원 수준이라 현실적으로 불가능해요.
결국 “돈이 안 맞아서”가 제일 큰 벽입니다.
"""

result = infer_with_weighted_score(
    text=text,
    tokenizer=tokenizer,
    model=model,
    classify_fn=classify_text,
    threshold=0.6
)

print("probs :", result["probs"])
print("pred  :", result["pred"])
print("score :", result["score"])

In [ ]:
text = """
심해도시는 보통 깊은 바다 밑이나 해저에 건설하는 거주 가능한 도시 구조물을 말합니다. SF에서 자주 등장하는 개념이지만, 실제로는 해저 기지나 수중 호텔 같은 소규모 시설만 존재하고 완전한 도시는 아직 기술적·경제적으로 실현이 어렵습니다. 단순히 "바다 밑 도시"로 이해해도 되지만, 실제 구현에는 극한 수압, 산소 공급, 자원 조달 등 복잡한 문제들이 있습니다.
가장 중요한 건 극한 수압을 견디는 구조물, 지속 가능한 산소 공급과 이산화탄소 제거 시스템, 그리고 식량·에너지 자급 능력입니다. 현재 기술로는 소규모 연구 기지나 잠수정 수준은 가능하지만(실제로 수중 연구소들이 운영된 적 있음), 장기간 대규모 거주는 비용과 안전성 문제로 여전히 어렵습니다. 국제우주정거장보다 접근성은 좋지만 유지비용과 위험도는 비슷하거나 더 높은 수준입니다.
심해는 우주보다 접근은 쉽지만 수압·부식·어둠 등 극한 환경이 지속되고, 건설·유지보수 비용이 막대하며, 비상 상황 시 탈출이 매우 어렵기 때문입니다. 땅 위에 도시를 짓는 것보다 수천 배 비싸면서도 얻을 수 있는 실질적 이득(자원, 공간 등)은 제한적이라 경제성이 없습니다. 결국 극한 환경 연구나 특수 목적 외에는 육지나 해상 부유도시가 훨씬 현실적인 대안으로 평가됩니다.
기술적으로는 이미 심해 잠수정, 해저 파이프라인 등으로 가능성이 입증됐지만, 결정적 장애물은 압도적인 건설·유지 비용과 위험 대비 실익 부족입니다. 수천 미터 깊이에서 대규모 구조물을 짓고 유지하는 비용은 천문학적이며, 사고 시 구조가 거의 불가능해 안전성 확보도 극히 어렵습니다. 결국 "기술적으로 할 수 있다"와 "실제로 할 만한 가치가 있다" 사이의 간극이 가장 큰 문제입니다.
"""

result = infer_with_weighted_score(
    text=text,
    tokenizer=tokenizer,
    model=model,
    classify_fn=classify_text,
    threshold=0.6
)

print("probs :", result["probs"])
print("pred  :", result["pred"])
print("score :", result["score"])

In [ ]:
text = """
**심해도시**는 주로 **수심 1,000m 이상의 깊은 바다에 건설되는 인공 거주/연구 시설**을 의미합니다. 단순한 해저 건축물이 아니라 생태계 보존, 자원 개발, 재난 대비 등을 위한 첨단 기술이 결합된 복합 공간으로 구상됩니다. SF 장르에서는 더 확장된 개념(예: 자급자족 도시)으로 묘사되기도 하지만, 현실적으로는 아직 초기 연구 단계입니다. 추가로 궁금한 점이 있다면 알려주세요! 🌊
**심해에서 장기 생활 시 핵심 조건은 압력 견디는 구조물, 산소·식량·물 공급 시스템, 심리적 스트레스 관리입니다.**
현재 기술로는 **단기 체류용 해저 기지**(예: 연구용 시설)는 가능하나, 자급자족형 도시는 에너지 효율·재료 내구성 등의 한계로 실현되지 않았습니다.
장기 생존을 위해선 **재생 에너지·폐쇄 생태계 기술** 발전이 필수적이며, 이는 아직 실험 단계에 머물러 있습니다.

궁금한 세부 사항이 있다면 추가로 알려드릴게요! 🤿
심해도시 실현이 어려운 이유는 **① 기술적 한계**(고압 견디는 초강력 소재, 지속 가능한 에너지원 부재), **② 천문학적 비용**(건설·유지보수), **③ 인간적 요인**(고립 스트레스, 사회 시스템 재구성 난제) 때문입니다.
현재 기술로는 단기 연구시설만 가능하며, 자급자족 도시화는 수십 년 이상 걸릴 전망입니다. 또한 해양 생태계 교란, 국제법적 분쟁 소지도 큰 장애물로 작용합니다. 🌊🔧
심해도시 실현의 가장 큰 장벽은 **기술적 한계**입니다. 고압 환경에 견딜 수 있는 초강력 소재 개발, 지속 가능한 에너지 공급 시스템 구축, 완전 폐쇄형 생태계 구현 등이 아직 해결되지 않았기 때문입니다. 물론 막대한 건설 비용과 안전 리스크도 현실적 문제이지만, 기술적 돌파구 없이는 근본적 진전이 불가능하다는 점에서 우선순위가 높습니다. 🛠️💡

"""

result = infer_with_weighted_score(
    text=text,
    tokenizer=tokenizer,
    model=model,
    classify_fn=classify_text,
    threshold=0.6
)

print("probs :", result["probs"])
print("pred  :", result["pred"])
print("score :", result["score"])

## 생활 질문

In [ ]:
text = """
가까운 두 사람이 단둘이 남겨진 상황에서 소외감이나 불안함을 느끼는 것은 지극히 **자연스러운 본능**이며, 결코 예민한 것이 아닙니다.

불편한 마음을 억누르기보다 "내가 그만큼 두 사람을 아끼고 있구나"라고 자신의 감정을 먼저 인정해 주는 것이 중요합니다.

계속 신경 쓰인다면 혼자 앓기보다 여자친구에게 "그때 소외감을 느껴서 조금 신경 쓰였다"라고 **가볍고 솔직하게 대화**하며 마음을 해소해 보세요.


혹시 여자친구에게 이 감정을 어떻게 꺼내야 할지 막막하시다면, 대화의 물꼬를 트는 부드러운 말투를 제안해 드릴까요?

상대를 의심해서가 아니라 **나의 심리적 경계선**을 확인하는 과정이라 생각하며, '사건'이 아닌 '내 기분'에 초점을 맞춰 보세요.

"너희를 믿지만 상황이 이렇다 보니 마음이 조금 허전했다"처럼 **'나'를 주어로 표현(I-Message)**하면 상대를 비난하지 않고도 진솔하게 전달됩니다.

이번 기회에 서로가 편안함을 느끼는 술자리 에티켓에 대해 가볍게 대화 나누며 **신뢰의 기준**을 함께 세워가는 계기로 삼는 것이 좋습니다.


여자친구가 오해하지 않도록 담백하게 마음을 전할 수 있는 구체적인 문구 예시를 몇 가지 만들어 드릴까요?
네, 맞습니다. 연인 사이의 **정서적 안전거리**를 지키고 싶은 본능적 경계심과 "나를 더 배려해줬으면" 하는 기대감이 부딪히며 생기는 자연스러운 신호입니다.

이러한 경계는 관계를 보호하려는 건강한 마음에서 비롯된 것이니, 스스로를 예민하다고 자책하기보다 **본인의 연애 가치관을 확인하는 과정**으로 받아들이셔도 좋습니다.

불편함의 원인을 알게 된 만큼, 내가 어느 정도 선까지를 존중받고 싶은지 스스로 정리해 본 뒤 파트너와 공유하면 관계가 훨씬 단단해질 것입니다.

---

본인이 생각하는 '적절한 경계선'이 어디까지인지 정리해보고 싶으신가요? 상황별 예시를 통해 함께 확인해 드릴 수 있습니다.

"그날 내가 먼저 자러 가고 나서 둘이 남겨지니, 너를 믿는 것과는 별개로 마음이 조금 쓸쓸하고 신경 쓰이더라"라고 **내 감정만 담백하게** 전해보세요.

비난이 아니라 **"우리의 건강한 관계를 위해 내 솔직한 마음을 공유하고 싶다"**는 점을 먼저 언급하면 상대도 방어적이지 않게 받아들일 수 있습니다.

"앞으로는 이런 상황이 생기면 서로 조금만 더 배려해주면 좋겠다"는 **미래 지향적인 부탁**으로 대화를 마무리하는 것이 가장 깔끔합니다.

---

여자친구의 예상 반응에 따라 대화를 이어가는 방법도 함께 고민해 드릴까요?

"""

result = infer_with_weighted_score(
    text=text,
    tokenizer=tokenizer,
    model=model,
    classify_fn=classify_text,
    threshold=0.6
)

print("probs :", result["probs"])
print("pred  :", result["pred"])
print("score :", result["score"])

In [ ]:
text = """
이건 예민하다기보다 **자연스러운 불편함**이야.
중요한 건 그 감정을 부정하지 말고, 왜 불편한지 스스로 정리한 뒤 차분하게 표현하는 거야.
신뢰가 핵심이니, 감정만 솔직히 나누고 책임 추궁은 피하는 게 좋아.
의심이 아니라 **내 감정이 불편했다**는 관점으로 상황을 바라보는 게 좋아.
사실 확인보다 신뢰를 전제로, 혼자 해석을 키우지 않도록 거리 두고 정리해봐.
필요하면 차분하게 감정만 공유하고, 판단이나 추궁은 보류하는 게 관계에 도움이 돼.
그럴 가능성 충분히 있어.
연인 관계에서의 경계나 기대가 건드려졌을 때 자연스럽게 불편함이 생겨.
이 기회에 나에게 중요한 기준이 뭔지 스스로 정리해보는 게 도움이 돼.
상대의 행동이 아니라 **내가 느낀 감정**에 초점을 맞춰 말해봐.
“그때 네가 잘못했다” 대신 “그 상황이 나는 조금 불편했어”처럼 표현하는 게 좋아.
해결 요구보단 이해를 구하는 톤이 관계를 지켜줘.
"""

result = infer_with_weighted_score(
    text=text,
    tokenizer=tokenizer,
    model=model,
    classify_fn=classify_text,
    threshold=0.6
)
print("probs :", result["probs"])
print("pred  :", result["pred"])
print("score :", result["score"])

In [ ]:
text = """
이 감정은 자연스러운 거야. 관계에서 신뢰가 중요하지만, 이런 상황에서 약간의 불안이나 질투가 생기는 건 흔한 일이야.
너무 예민하다고 자책하지 말고, 여자친구와 솔직하게 이야기해보는 게 좋을 수 있어.
신뢰를 쌓아가며 넘기면 마음이 편해질 거야.
자연스러운 감정이라고 인정하고, “그날 이후로 좀 신경 쓰이네” 정도로 가볍게 솔직히 털어놓는 게 제일 좋아.
의심보다는 “나 이런 마음이 들더라”는 식으로 말하면 관계가 오히려 더 돈독해질 수 있어.
그 반응 보면서 천천히 정리해도 괜찮아.
네, 대부분 그런 불편함은 연인 관계에서 자연스럽게 생기는 '경계'나 '독점적 기대' 때문이야.
그게 잘못된 게 아니라 인간적인 감정이고, 오히려 관계를 소중히 여긴다는 증거일 수 있어.
그걸 인정하는 순간 마음이 좀 더 가벼워질 거야.
“요즘 그날 밤 생각나면 좀 마음이 불편하더라… 너 믿는데도 괜히 신경 쓰여서 말해봤어.”
이렇게 ‘내 감정’ 중심으로 부드럽게 시작하면 따지는 느낌 없이 솔직하게 전달돼.
그 다음엔 “우리 더 편하게 지내고 싶어서 말한 거야”라고 마무리하면 오해도 줄어들어.
"""

result = infer_with_weighted_score(
    text=text,
    tokenizer=tokenizer,
    model=model,
    classify_fn=classify_text,
    threshold=0.6
)

print("probs :", result["probs"])
print("pred  :", result["pred"])
print("score :", result["score"])

In [ ]:
text = """
불편한 감정은 충분히 자연스러운 반응이에요. 신뢰 관계에서 오는 감정일 수도 있고, 단순히 상황이 신경 쓰여서일 수도 있죠.

가장 좋은 건 여자친구에게 솔직하게 "그날 상황이 계속 마음에 걸려서"라고 편하게 이야기해보는 거예요. 대화를 통해 서로의 생각을 확인하면 불필요한 오해나 불안감을 덜 수 있을 거예요.
의심이 아니라 "내 감정"을 중심으로 대화하면 관계가 상하지 않아요. "너희가 뭘 했다"가 아니라 "나는 이런 게 좀 신경 쓰였어"라고 말하는 거죠.
여자친구에게 가볍게 "그날 둘이 더 놀았다는 게 좀 이상하게 느껴졌어. 나만 그런가?"라고 물어보세요. 상대방 반응을 보면서 불편함의 원인이 내 불안인지, 실제 문제가 있는지 구분할 수 있을 거예요.
맞아요, 사람마다 연인 관계에서 편하게 느끼는 경계가 달라요. 누군가는 괜찮아도, 당신에게는 불편할 수 있고 그건 잘못된 게 아니에요.
중요한 건 상대방도 당신의 경계를 알아야 한다는 거예요. "나는 이런 상황이 좀 불편하더라"고 말하면, 서로의 기준을 맞춰갈 수 있고 앞으로 비슷한 일을 피할 수 있어요.
"너희를 의심하는 건 아닌데, 그날 일이 자꾸 마음에 걸려서 말하고 싶었어"라고 시작하면 방어적인 분위기를 피할 수 있어요.
"나만 먼저 자고 너희 둘이 계속 있었던 게 생각보다 신경 쓰이더라. 이런 감정이 드는 게 이상한 건지 네 생각도 듣고 싶어"처럼 질문 형태로 열어두면, 대화가 자연스럽게 이어질 거예요.

"""

result = infer_with_weighted_score(
    text=text,
    tokenizer=tokenizer,
    model=model,
    classify_fn=classify_text,
    threshold=0.6
)

print("probs :", result["probs"])
print("pred  :", result["pred"])
print("score :", result["score"])

In [26]:
text = """
당신의 불편함은 자연스러운 감정입니다. 연인과 가까운 친구 간의 관계에서 경계심이나 질투는 흔히 나타나는 감정이에요. 이를 억누르기보다 **직접 솔직하게 대화**해보는 것이 좋습니다. 예를 들어 "너희끼리 술 마셔서 조금 신경 쓰였어"라며 편안한 분위기에서 털어놓으면 서로의 생각을 확인하고 신뢰를 다질 기회가 될 거예요. 감정을 부정하지 말고, 오히려 관계 개선을 위한 계기로 삼아보세요.
관계에 틈이 생기지 않도록 하려면 감정을 관찰하되 섣불리 판단하지 않는 것이 중요합니다. 우선 두 사람의 평소 행동과 이번 상황을 분리해 생각해보세요. 예를 들어, 친구와 여자친구가 평소에도 친밀했다면 단순한 우정일 수 있지만, 최근 변화된 점이 있다면 주의 깊게 살펴보는 게 좋아요. 이후 자연스러운 대화 기회를 잡아 "너희가 같이 있는 모습이 눈에 띄더라"며 편안하게 털어놓고, 상대방의 반응을 보며 경계를 조절해가세요. 의심보다는 호기심을 갖고 접근하면 관계 손상 없이 진실을 파악할 수 있을 거예요.
네, 그럴 가능성이 높습니다. 연인 관계에서 **"특별함"이나 "우선순위"**에 대한 기대가 있다면, 친구와의 친밀한 모습이 위협적으로 느껴질 수 있어요. 또한, **경계 설정(예: 함께 있는 시간, 공유하는 비밀)**이 모호하다면 불안감이 커지기 쉽죠.
해결책은 파트너와 솔직한 대화입니다. "너와 친구 사이가 가까워 보여서 조금 신경 쓰여"라며 감정을 전달하고, 서로의 선과 기대를 조율해보세요. 건강한 경계는 오히려 관계를 단단히 만듭니다.
여자친구에게 말할 때는 **"I-메시지"**로 시작하세요. 예를 들어, *"내가 그때 너희 둘이 남아서 술 마시는 걸 보니 조금 외로운 기분이 들었어"*처럼 자신의 감정에 초점을 맞추어 전달하세요. 상대방을 비난하거나 추궁하지 말고, 호기심 어린 질문으로 이어가는 것도 좋아요. *"다음에는 내가 함께할 수 있는 시간을 미리 계획해보면 어때?"*라며 미래 지향적인 해결책을 제안하면 관계가 더 돈독해질 거예요.

핵심은 감정 공유 → 이해 요청 → 협력적 해결 단계로 자연스럽게 흘러가게 하는 겁니다!
"""

result = infer_with_weighted_score(
    text=text,
    tokenizer=tokenizer,
    model=model,
    classify_fn=classify_text,
    threshold=0.6
)
print("probs :", result["probs"])
print("pred  :", result["pred"])
print("score :", result["score"])

probs : [0.966  0.901  0.9805 0.92   0.8115 0.841  0.9673 0.9604 0.987 ]
pred  : [1 1 1 1 1 1 1 1 1]
score : 9.168
